In [1]:
!curl localhost:9200

{
  "name" : "8c3059a8b835",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "F6FC4phYTguOu1H9YGR_Dg",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [2]:
import requests

docs_url = "https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1"
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course["course"]

    for doc in course["documents"]:
        doc["course"] = course_name
        documents.append(doc)

In [3]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [4]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch("http://localhost:9200/")
es_client.info()

ObjectApiResponse({'name': '8c3059a8b835', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'F6FC4phYTguOu1H9YGR_Dg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [ ]:
index_prop = {
    "properties": {
        "text": {"type": "text"},
        "section": {"type": "text"},
        "question": {"type": "text"},
        "course": {"type": "keyword"},
    }
}

es_client.indices.create(index="course-faq", mappings=index_prop)

In [6]:
from tqdm.notebook import tqdm

for doc in tqdm(documents):
    es_client.index(index="course-faq", document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [7]:
q = "How do I execute a command in a running docker container?"

In [11]:
query = {
    "multi_match": {"query": q, "fields": ["text", "question^4"], "type": "best_fields"}
}

res = es_client.search(index="course-faq", query=query)

In [13]:
res["hits"]["max_score"]

84.050095

In [30]:
query2 = {
    "bool": {
        "must": {
            "multi_match": {
                "query": q,
                "fields": ["text", "question^4"],
                "type": "best_fields",
            }
        },
        "filter": {"term": {"course": "machine-learning-zoomcamp"}},
    }
}

res2 = es_client.search(index="course-faq", query=query2)

In [32]:
match = [x["_source"] for x in res2["hits"]["hits"]][:3]
match

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

In [34]:
context_template = """
Q: {question}
A: {text}
""".strip()

context_list = [
    context_template.format(question=x["question"], text=x["text"]) for x in match
]
context = "\n\n".join(context_list)
context

'Q: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I copy files from my local machine to docker container?\nA: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani\n\nQ: How do I copy files from a different folder into docker container’s working directory?\nA: You can copy files from your local machine into a Docker container using the docker cp comman

In [35]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

q = "How do I execute a command in a running docker container?"

prompt = prompt_template.format(question=q, context=context)
len(prompt)

1462

In [39]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")
len(encoding.encode(prompt))

322